# 기출 9회_작업형2
## 농작물에서 농약 검출 여부 예측
-	제공된 데이터 목록:
    - farm_train.csv (훈련 데이터)
    - farm_test.csv (평가용 데이터)
-	예측할 컬럼: 농약검출여부 (0:미검출, 1:검출, 2:재검사 필요)

학습용 데이터(farm_train.csv)를 이용하여 농약 검출 여부를 예측하는 모델을 구축한 후, 이를 평가용 데이터(farm_test.csv)에 적용하여 예측 결과를 다음과 같은 형식의 CSV 파일로 생성하시오.

제출 파일은 다음 한 개의 컬럼을 포함해야 합니다.
- pred: 예측된 검출 여부
- 제출 파일명: 'result.csv'

모델의 성능은 macro F1 Score 평가지표에 따라 채점됩니다.
제출 CSV 파일은 result.csv 형태로 제출하시오.

~~~
pred
0
1
0
2
0
...
~~~

## 📌 My Code
- **lightgbm은 sklearn 안에 없고, 바로 import 해줘야 함. => 패키지 설치되어 있지 않아서 실전에서 활용 불가능할 듯**
- **RandomForestClassifier**: 0.8643039773063413
- **HistGradientBoostingClassifier**: 0.9336837464154217

In [42]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_test.csv")

# 1. 데이터 유형 확인
# 결측치: 없음
# 범주형: ['지역', '작물종류', '토양유형', '등급'] => '연도'를 범주로 변환해주면 더 성능이 좋을까? 0.86792 (기존: 0.81631)
# target: 농약검출여부
# print(train.info())
# print(test.info())
# print(train.shape, test.shape) # (4000, 9) (1000, 8)
# print(train.select_dtypes('object').columns)

# 2. 데이터 전처리
# (1) X_full
X = train.drop(['농약검출여부'], axis=1)
y = train['농약검출여부']
X_full = pd.concat([X, test], axis=0)

# '연도'를 범주형으로 바꿔보기!
X_full['연도'] = X_full['연도'].astype('str')
print(X_full.select_dtypes('object').columns)

# print(X_full.shape, X.shape, test.shape) # (5000, 8) (4000, 8) (1000, 8)

# (2) 결측치 처리 => Pass

# (3) 수치형 스케일링 => Pass

# (4) 범주형 인코딩
X_full = pd.get_dummies(X_full)
# print(X_full)
# print(X_full.shape) # (5000, 28)

# 3. 데이터 분리
X = X_full[:train.shape[0]]
X_test = X_full[train.shape[0]:]
# print(X.shape, X_test.shape) # (4000, 28) (1000, 28)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape) # (3200, 28) (800, 28) (3200,) (800,)

# 4. 모델 학습 및 검증
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
# print(y_val_pred)
# print(y_val_pred.shape) # (800,)

from sklearn.ensemble import HistGradientBoostingClassifier
model2 = HistGradientBoostingClassifier()
model2.fit(X_train, y_train)
y_val_pred2 = model2.predict(X_val)
# print(y_val_pred2)
# print(y_val_pred2.shape) # (800,)

# 5. 모델 평가
from sklearn.metrics import f1_score
f1_score_1 = f1_score(y_val, y_val_pred, average='macro')
print('RandomForestClassifier', f1_score_1)

f1_score_2 = f1_score(y_val, y_val_pred2, average='macro')
print('HistGradientBoostingClassifier', f1_score_2)

# 6. 결과 저장
y_pred = model.predict(X_test)
# print(y_pred)
# print(y_pred.shape) # (1000,)

result = pd.DataFrame({'pred':y_pred})
result.to_csv('result_92.csv', index=False)

check = pd.read_csv('result_92.csv')
# print(check)

Index(['연도', '지역', '작물종류', '토양유형', '등급'], dtype='object')
RandomForestClassifier 0.8643039773063413
HistGradientBoostingClassifier 0.9336837464154217


# 작업형2 풀이

## 문제정의

In [ ]:
# 문제정의
# 평가: f1-macro
# target: 농약검출여부
# 최종파일: result.csv(컬럼 1개 pred)

## 데이터 불러오기

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_test.csv")

## 탐색적 데이터 분석(EDA)

In [ ]:
# 데이터 크기
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

Train Shape: (4000, 9)
Test Shape: (1000, 8)


In [ ]:
# 데이터 샘플
train.head()

,농업면적,연도,지역,비료사용량,비료잔여량,작물종류,토양유형,농약검출여부,등급
0,20079.652837,2004,대구,407.985516,146.290507,보리,점토,2,C
1,73858.643204,2012,울산,221.229692,1967.333638,밀,점토,0,B
2,65718.150861,2012,강원,370.967205,2253.522610,쌀,점토,0,B
3,37366.182902,2005,광주,274.128236,1487.535265,쌀,양토,0,B
4,81515.151289,2007,충남,213.410655,683.306745,쌀,양토,1,B


In [ ]:
# 데이터 정보(자료형)
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   농업면적    4000 non-null   float64
 1   연도      4000 non-null   int64  
 2   지역      4000 non-null   object 
 3   비료사용량   4000 non-null   float64
 4   비료잔여량   4000 non-null   float64
 5   작물종류    4000 non-null   object 
 6   토양유형    4000 non-null   object 
 7   농약검출여부  4000 non-null   int64  
 8   등급      4000 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 281.4+ KB
None


In [ ]:
# object컬럼 unique 수
# train
print(train.describe(include='object'))

# test
print(test.describe(include='object'))

          지역  작물종류  토양유형    등급
count   4000  4000  4000  4000
unique    15     3     3     3
top       전북    보리    양토     A
freq     302  1374  1357  1364
          지역  작물종류  토양유형    등급
count   1000  1000  1000  1000
unique    15     3     3     3
top       대전     쌀    양토     A
freq      84   352   347   336


In [ ]:
# 결측치 수
# train
print(train.isnull().sum().sum())

# test
print(test.isnull().sum().sum())

0
0


In [ ]:
# target unique 수
print(train['농약검출여부'].value_counts())

농약검출여부
2    1989
0    1758
1     253
Name: count, dtype: int64


## 데이터 전처리

In [ ]:
# 타겟 옮기기
target = train.pop('농약검출여부')

In [ ]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

## 검증데이터 나누기

In [ ]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((3200, 28), (800, 28), (3200,), (800,))

## 모델 학습 및 평가

In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

In [ ]:
# macro F1 score
from sklearn.metrics import f1_score
print("랜덤포레스트")
print(f1_score(y_val, pred, average='macro'))

랜덤포레스트
0.8532014300116062


In [ ]:
# LightGBM
import lightgbm as lgb
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print("lightgbm")
print(f1_score(y_val, pred, average='macro'))

lightgbm
0.9100316620356779


In [ ]:
# 최종 제출 파일 (lightGBM)
pred = lg.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv("result.csv", index=False)

### 꼭 확인할 내용

In [ ]:
# 1. pred 행의 수와 test의 행의 수 비교
print("pred: ",pred.shape) # test 행의 수: 1000

# 2. 생성한 csv 확인
print(pd.read_csv("result.csv").head())

pred:  (1000,)
   pred
0     2
1     0
2     0
3     2
4     0


## 위 코드 한번에 작성하기

In [ ]:
# 문제정의
# 평가: f1-macro
# target: 농약검출여부
# 최종파일: result.csv(컬럼 1개 pred)

# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_test.csv")

# 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n ===== 데이터 샘플 =====")
print(train.head())

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train object컬럼 unique 수 =====")
print(train.describe(include='object'))

print("\n ===== test object컬럼 unique 수 =====")
print(test.describe(include='object'))

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== target unique 수 =====")
print(train['농약검출여부'].value_counts())

===== 데이터 크기 =====
Train Shape: (4000, 9)
Test Shape: (1000, 8)

 ===== 데이터 샘플 =====
           농업면적    연도  지역       비료사용량        비료잔여량 작물종류 토양유형  농약검출여부 등급
0  20079.652837  2004  대구  407.985516   146.290507   보리   점토       2  C
1  73858.643204  2012  울산  221.229692  1967.333638    밀   점토       0  B
2  65718.150861  2012  강원  370.967205  2253.522610    쌀   점토       0  B
3  37366.182902  2005  광주  274.128236  1487.535265    쌀   양토       0  B
4  81515.151289  2007  충남  213.410655   683.306745    쌀   양토       1  B

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   농업면적    4000 non-null   float64
 1   연도      4000 non-null   int64  
 2   지역      4000 non-null   object 
 3   비료사용량   4000 non-null   float64
 4   비료잔여량   4000 non-null   float64
 5   작물종류    4000 non-null   object 
 6   토양유형    4000 non-null   object 
 7   농약검출여부  4000 

In [ ]:
# 데이터 다시 불러오기
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_2/farm_test.csv")

# 데이터 전처리
target = train.pop('농약검출여부')

# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

# macro F1 score
from sklearn.metrics import f1_score
print("랜덤포레스트")
print(f1_score(y_val, pred, average='macro'))

# LightGBM
import lightgbm as lgb
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print("lightgbm")
print(f1_score(y_val, pred, average='macro'))

# 최종 제출 파일 (lightGBM)
pred = lg.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)


랜덤포레스트
0.8532014300116062
lightgbm
0.9100316620356779


In [ ]:
# 1. pred 행의 수와 test의 행의 수 비교
print("pred: ",pred.shape) # test 행의 수: 1000

# 2. 생성한 csv 확인
print(pd.read_csv("result.csv").head())

pred:  (1000,)
   pred
0     2
1     0
2     0
3     2
4     0
